In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import hydromt
from os.path import join, basename, dirname
import matplotlib.pyplot as plt
import matplotlib as mpl
from hydromt_sfincs import SfincsModel
import subprocess
import os
import geopandas as gpd

In [2]:
# simulations
scens = pd.read_csv(join(rdir, 'sim_SCEN.csv'), index_col=0).rename(columns={'h_tsw_rp': 'h_rp'})
print(len(scens.index))
scens[500:].head(10)

NameError: name 'rdir' is not defined

In [2]:
# read events
ddir = r'../../1_data/2_forcing'
ds_q = xr.open_dataset(join(ddir, r'cama_discharge_beira_daily_events.nc'))
ds_q = np.maximum(ds_q, ds_q.sel(rps=0).max())
da_h = xr.open_dataarray(join(ddir, r'reanalysis_gtsm_v1_beira_extended_events.nc'))
da_p = xr.open_dataarray(join(ddir, r'era5_precip_beira_hourly_spatialmean_events.nc'))
events = dict(
    h = da_h.reset_coords(drop=True),
    qb = ds_q['qb'].reset_coords(drop=True),
    qp = ds_q['qp'].reset_coords(drop=True),
    p = da_p.reset_coords(drop=True),
)

In [3]:
# lagtimes relative to qb
from datetime import timedelta
rdir = r'../../4_results'
lagtimes = pd.read_csv(join(rdir, 'lagtimes.csv'), index_col=0)['lag'].to_dict()
lagtimes = {k: timedelta(days=v) for k,v in lagtimes.items()}
lagtimes['h'] = lagtimes['s']
lagtimes['qb'] = timedelta(days=0)
lagtimes

{'qp': datetime.timedelta(days=1),
 'p': datetime.timedelta(days=-3),
 's': datetime.timedelta(days=-3),
 'w': datetime.timedelta(days=-3),
 'h': datetime.timedelta(days=-3),
 'qb': datetime.timedelta(0)}

In [ ]:
from hydromt_sfincs.utils import parse_datetime, write_timeseries, write_inp

tstart = '20200101 000000'
tstop = '20200115 000000'
tref = parse_datetime(tstart)
t0 = tref + timedelta(days=7)

def get_ts(dvar, rp, lagtimes=lagtimes, events=events):
    if rp not in events[dvar].rps:
        return
    ts = events[dvar].sel(rps=rp).to_series()
    ts.index += (lagtimes[dvar].total_seconds() / 3600)
    ts.index = t0 + np.array([timedelta(hours=dt) for dt in ts.index.values])
    dates = pd.date_range(tstart, tstop, freq=np.diff(ts.index.to_pydatetime())[0])
    ts = ts.reindex(dates, fill_value=0)
    return ts

In [4]:
from typing import Tuple, Union
from pathlib import Path

def read_binary_output(
    fn: Union[str, Path],
    ind: np.ndarray,
    timesteps: int,
    shape: Tuple[int],
    mv: float = -999.0,
    dtype: str = "f4",
) -> np.ndarray:
    """Read binary map.

    Parameters
    ----------
    fn: str, Path
        Path to map file.
    ind: np.ndarray
        1D array of flat index of binary maps.
    timesteps: int
        Numer of timesteps in the output file
    shape: tuple of int
        (nrow, ncol) shape of output map.
    mv: int or float
        missing value, by default -999.0.
    dtype: str, np.dtype, optional
        Data type, by default "f4".

    Returns
    -------
    ind: np.ndarray
        1D array of flat index of binary maps.
    """
    assert ind.max() <= np.multiply(*shape)
    nrow, ncol = shape
    data = np.full((timesteps, ncol, nrow), mv, dtype=dtype)
    with open(fn, 'r') as fid:
        for it in range(timesteps):
            d0 = np.full((ncol, nrow), mv, dtype=dtype)
            d0.flat[ind] = np.fromfile(fid, dtype=dtype, count=ind.size+2)[1:-1]
            data[it,:,:] = np.fliplr(d0)
    return data.transpose(0,2,1)

In [5]:
from hydromt_sfincs import utils

def postprocess(root, min_flddph=0):
    # open model instance
    mod = SfincsModel(root, mode='r')

    # get model props
    nrow, ncol = mod.config['nmax'], mod.config['mmax']
    tstart, tstop = mod.get_model_time()
    dt = timedelta(seconds=mod.config['dtmaxout'])
    time = pd.date_range(tstart+dt, end=tstop, freq=dt)

    # read zsmax
    ind = utils.read_binary_map_index(join(root, mod.config['indexfile']))
    da = xr.DataArray(
        data = read_binary_output(join(root, 'zsmax.dat'), ind, time.size, (nrow, ncol)),
        dims = ('timemax', 'y', 'x'),
        coords = {'timemax': time, **mod.staticmaps.raster.coords}
    )
    da_dep = mod.staticmaps['dep']

    # calculate hmax and write to raster
    hmax = (da-da_dep).where(da!=-999,0).where(da_dep!=da_dep.raster.nodata)
    hmax_tot = hmax.max('timemax')
    
    if not os.path.isfile(join(root, 'gis', 'hmax.tif')):
        hmax_tot.raster.set_nodata(-999)
        hmax_tot.raster.set_crs(mod.crs)
        mod.set_results(hmax_tot.fillna(-999), 'hmax')
        mod.write_raster('results.hmax')

    # plot hmax
    fig, ax = mod.plot_basemap(fn_out=None, variable='', geoms=[], plot_bounds=False, bmap='StamenTerrain')
    hmax_tot.where(hmax_tot>min_flddph).plot(
        ax=ax, cmap='viridis', vmin=min_flddph, vmax=3, cbar_kwargs=dict(shrink=0.5, label='flood depth [m]')
    )
    ax.set_title(os.path.basename(root))
    if not os.path.isdir(join(root, 'figs')):
        os.makedirs(join(root, 'figs'))
    plt.savefig(join(root, 'figs', f'hmax.png'), dpi=150)
    plt.close()

    # remove output binary files
    for fn in glob.glob(join(root, '*.dat')):
        try:
            os.unlink(fn)
        except:
            pass

## prepare basemodel

Execute sfincs.bat in ../1_prepare

## events


In [6]:
# read basemodel
mdir = r"../../3_models/sfincs"
basename = '00_base_riv'
mod0 = SfincsModel(join(mdir, basename), mode='r+')

In [ ]:
# select buzi and pungwe rivers
src = mod0.forcing['dis'].vector.to_gdf().loc[[1,4],:]
# h boundary location based on gtsm output location
bnd = gpd.GeoDataFrame(
    index=np.atleast_1d(da_h['stations'].values),
    geometry=gpd.points_from_xy(
        np.atleast_1d(da_h['station_x_coordinate'].values), 
        np.atleast_1d(da_h['station_y_coordinate'].values)
    ),
    crs=4326
).to_crs(src.crs)

In [ ]:
# modify config
config = mod0.config.copy()
config.update({
    'tref': tstart, 
    'tstart': tstart, 
    'tstop': tstop, 
    'outputformat': 'bin',
    'dtmaxout': 86400,
    'dtwnd': 600,
    'alpha': 0.7,
    'precipfile': 'sfincs.precip',
    'bzsfile': 'sfincs.bzs',
    'bndfile': 'sfincs.bnd',
    'inifile': '../qb000_qp000_h000_p000/sfincs.zsini'
})
config.pop('netamprfile',None)
# config

In [ ]:
for i, run in scens.iterrows():
    name = run['scen']
    root = join(mdir, name)

    if os.path.isfile(join(root, 'sfincs.inp')): continue
    print(f'>>{name}')
    # mod0 = SfincsModel(join(mdir, basename), mode='r+')
    mod0.set_root(root, mode='w')
    mod0.setup_config(**config)
    mod0.config.pop('restartfile', None)
    if np.all(run[:4]==0):
        mod0.setup_config(restartfile=mod0.config.pop('inifile'))

    qb0 = get_ts('qb', run['qb_rp']).rename(1)
    qp0 = get_ts('qp', run['qp_rp']).rename(4)
    q0 = pd.concat([qb0, qp0], axis=1)
    mod0.set_forcing_1d(ts=q0, xy=src, name='discharge')
    
    h0 = get_ts('h', run['h_rp']).rename(bnd.index.item()).to_frame()
    mod0.set_forcing_1d(ts=h0, xy=bnd, name='waterlevel')
    
    p0 = get_ts('p', run['p_rp'])
    if p0 is not None:
        mod0.set_forcing_1d(ts=p0, xy=None, name='precip')
    else:
        mod0.forcing.pop('precip', None)
        mod0.config.pop('precipfile')
    
    mod0.write_forcing()
    mod0.write_config(rel_path=f'../{basename}')
    
    # mod0.plot_forcing()
    # plt.close('all')
    # mod0.plot_basemap()
    # break

In [7]:
# run models
import subprocess
from os.path import dirname, isfile
import glob

fn_exe = "p:/11205283-hydromt-floodmodelling/02_models/bin/subgrid_openacc_11_rev295_16092021/sfincs.exe"

def check_finished(root):
    finished = False
    failed = False
    if isfile(join(root, 'sfincs.log')):
        with open(join(root, 'sfincs.log'), 'r') as f:
            for l in f.readlines():
                if 'Simulation stopped' in l:
                    print(f'FAILED: {root}')
                    failed = True
                    break
                elif 'Simulation is finished' in l:
                    finished = True
                    break
        if failed:
            os.rename(join(root, 'sfincs.log'), join(root, 'sfincs.log.failed'))
    return finished

runs = [dirname(fn) for fn in glob.glob(join(mdir, '*', 'sfincs.inp')) if not check_finished(dirname(fn))]
n = len(runs)
print(n)
for i, run in scens.iterrows():
    name = run['scen']
    root = join(mdir, name)
    if root not in runs or isfile(join(root, "sfincs.log")): 
        continue
    # mod = SfincsModel(root, 'r+')
    # mod.read_config()
    # mod.setup_config({'dtout':86400})
    # mod.write_config()
    print(f'{i+1:d}: {name}')
    with open(join(root, "sfincs.log"), 'w') as f:
        p = subprocess.Popen([fn_exe], stdout=f, cwd=root)
        %time p.wait()
        postprocess(root)

PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: '../../3_models/sfincs\\qb002_qp025_h002_p002\\sfincs.log' -> '../../3_models/sfincs\\qb002_qp025_h002_p002\\sfincs.log.failed'